# Tutorial for DLD sectors alignment using photon peak

## Preparation

### Import necessary libraries

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import numpy as np

from sed import SedProcessor
from sed.dataset import dataset

import matplotlib.pyplot as plt
%matplotlib widget

from lmfit.models import GaussianModel

### Get data paths

If it is your beamtime, you can read the raw data and write to the processed directory. For the public data, you can not write to the processed directory.

The paths are such that if you are on Maxwell, it uses those. Otherwise, data is downloaded in the current directory from Zenodo:
https://zenodo.org/records/15011781

In [ ]:
beamtime_dir = "/asap3/flash/gpfs/pg2/2021/data/11010004" # on Maxwell
if os.path.exists(beamtime_dir) and os.access(beamtime_dir, os.R_OK):
    path = beamtime_dir + "/raw/hdf/FL1USER3"
    buffer_path = beamtime_dir + "/processed/tutorial/"
else:
    # data_path can be defined and used to store the data in a specific location
    dataset.get("Photon_peak") # Put in Path to a storage of at least 10 GByte free space.
    path = dataset.dir
    buffer_path = path + "/processed/"

### Config setup

Here, we get the path to the config file and set up the relevant directories. This can also be done directly in the config file.

In [ ]:
# pick the default configuration file for hextof@FLASH
config_file = Path('../src/sed/config/flash_example_config.yaml')
assert config_file.exists()

In [ ]:
# here we setup a dictionary that will be used to override the path configuration
# a few setting changes are needed as well to work with older data
config_override = {
    "core": {
        "beamtime_id": 11010004,
        "paths": {
            "raw": path,
            "processed": buffer_path
        },
    },
    "dataframe": {
        "ubid_offset": 0,
        "channels": {
            "timeStamp": {
                "index_key": "/zraw/TIMINGINFO/TIME1.BUNCH_FIRST_INDEX.1/dGroup/index",
                "dataset_key": "/zraw/TIMINGINFO/TIME1.BUNCH_FIRST_INDEX.1/dGroup/time",
            },
            "pulseId": {
                "index_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/index",
                "dataset_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/value",
            },
            "dldPosX": {
                "index_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/index",
                "dataset_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/value",
            },
            "dldPosY": {
                "index_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/index",
                "dataset_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/value",
            },
            "dldTimeSteps": {
                "index_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/index",
                "dataset_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/value",
            },
            "dldAux": {
                "index_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/index",
                "dataset_key": "/zraw/FLASH.FEL/HEXTOF.DAQ/DLD1/dGroup/value",
            },
            "bam": {
                "index_key": "/zraw/FLASH.SDIAG/BAM.DAQ/4DBC3.HIGH_CHARGE_ARRIVAL_TIME/dGroup/index",
                "dataset_key": "/zraw/FLASH.SDIAG/BAM.DAQ/4DBC3.HIGH_CHARGE_ARRIVAL_TIME/dGroup/value",
            },
            "delayStage": {
                "index_key": "/zraw/FLASH.SYNC/LASER.LOCK.EXP/FLASH1.MOD1.PG.OSC/FMC0.MD22.1.ENCODER_POSITION.RD/dGroup/index",
                "dataset_key": "/zraw/FLASH.SYNC/LASER.LOCK.EXP/FLASH1.MOD1.PG.OSC/FMC0.MD22.1.ENCODER_POSITION.RD/dGroup/value",
            },
            "opticalDiode": {
                "format": "per_train",
                "index_key": "/uncategorised/FLASH.LASER/FLACPUPGLASER1.PULSEENERGY/PG2_incoupl/PULSEENERGY.MEAN/index",
                "dataset_key": "/uncategorised/FLASH.LASER/FLACPUPGLASER1.PULSEENERGY/PG2_incoupl/PULSEENERGY.MEAN/value",
            },
        },
    },
}

### Read data

In [ ]:
run_number = 40887
sp_ph_peak = SedProcessor(runs=[run_number], config=config_override, system_config=config_file, verbose=True)
sp_ph_peak.add_jitter()

Check which channels are included in the dataframe

In [ ]:
sp_ph_peak.dataframe[["dldTimeSteps", "dldSectorID"]].head()

## Data w/o correction of quadrants in time

First, we take a look at the photon peak and apply separation by single quadrants before any corrections. We plot the data in detector time (dldTimeSteps) as well as in detector position (dldPosX and dldPosY) coordinates, with additional separation by single sectors.

In [ ]:
axes = ['dldSectorID', 'dldTimeSteps','dldPosX','dldPosY']
ranges = [[0,8], [2360,2460], [435,885], [445,895]]
bins = [8,700,225,225]
res_ph_peak = sp_ph_peak.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
res_ph_peak['dldPosX'].attrs['unit'] = 'pixel'
res_ph_peak['dldPosY'].attrs['unit'] = 'pixel'
fig,ax = plt.subplots(1,2,figsize=(6,2.25), layout='tight')
res_ph_peak.sum(('dldSectorID','dldPosX','dldPosY')).plot(ax=ax[0])
res_ph_peak.sel(dldTimeSteps=slice(2380,2400)).mean(('dldSectorID','dldTimeSteps')).plot(ax=ax[1], robust=True)

Just photon peak itself without surrounding background

In [ ]:
ph_peak = res_ph_peak.sel(dldTimeSteps=slice(2380,2400)).sum(('dldPosX','dldPosY'))
plt.figure(figsize=(6,4))
ph_peak.sum('dldSectorID').plot()

Let's check the signal (photon peak) from every single sector

In [ ]:
plt.figure(figsize=(6,4))
ph_peak.plot()

In [ ]:
plt.figure(figsize=(6,4))
for i, item in enumerate(ph_peak):
    item.plot(label=f'S{i}')
    plt.legend()

### Position of the photon peak

In [ ]:
Gauss_mod = GaussianModel()

x=ph_peak['dldTimeSteps']
y=ph_peak.sum('dldSectorID')

pars = Gauss_mod.make_params(amplitude=200, center=2390, sigma=1)
# pars = Gauss_mod.guess(y, x=x)
out = Gauss_mod.fit(y, pars, x=x)

print(out.fit_report())
plt.figure(figsize=(6,4))
plt.plot(x,y, 'rx')
plt.plot(x,out.best_fit, "b", label="FWHM = {:.3f}".format(out.values['fwhm']))
plt.title(f'Run {run_number}, full photon peak')
plt.legend(loc="best")
plt.xlabel("dldTimeSteps [step]")

Width of the photon peak from every quadrant and they offset in ps in respect to 0 quadrant

In [ ]:
plt.figure(figsize=(6,4))
sector_delays = np.zeros(8)
for i, item in enumerate(ph_peak):
    x=ph_peak['dldTimeSteps']
    y=item
    pars = Gauss_mod.make_params(amplitude=200, center=2390, sigma=1)
    out = Gauss_mod.fit(y, pars, x=x)
    Center = 2388.984276411258
    Diff = "{:.3f}".format(Center - out.values['center'])
    sector_delays[i] = (out.values['center'])
    FWHM = "{:.3f}".format(out.values['fwhm'])
    item.plot(label=f'S{i}={Diff}, FWHM = {FWHM}')
    plt.title(f'Run {run_number}, individual sectors, not aligned')
    plt.legend()

In [ ]:
sector_delays = sector_delays - np.mean(sector_delays)
sector_delays

### sector alignment
as usual, first, we jitter, but here we also align in time the 8 sectors of the dld. This is done by finding the time of the maximum of the signal in each sector, and then shifting the signal in each sector by the difference between the maximum time and the time of the maximum in each sector.

In [ ]:
sp_ph_peak.align_dld_sectors(sector_delays=sector_delays)

In [ ]:
sp_ph_peak.dataframe[["dldTimeSteps", "dldSectorID"]].head()

### Width of the photon peak after sector alignment
Now we can repeat the fit procedure for combined and sector-separated photon peaks to see the effect of sector alignment

In [ ]:
axes = ['dldSectorID', 'dldTimeSteps','dldPosX','dldPosY']
ranges = [[0,8], [2360,2460], [435,885], [445,895]]
bins = [8,700,225,225]
res_ph_peak_align = sp_ph_peak.compute(bins=bins, axes=axes, ranges=ranges)

ph_peak_align = res_ph_peak_align.sel(dldTimeSteps=slice(2380,2400)).sum(('dldPosX','dldPosY'))

fig,ax = plt.subplots(1,2,figsize=(6,3.25), layout='tight')
ph_peak_align.sum('dldSectorID').plot(ax=ax[0])
for i, item in enumerate(ph_peak_align):
    item.plot(ax=ax[1], label=f'S{i}')
    plt.legend()

In [ ]:
Gauss_mod = GaussianModel()

x=ph_peak_align['dldTimeSteps']
y=ph_peak_align.sum('dldSectorID')

pars = Gauss_mod.make_params(amplitude=200, center=2390, sigma=1)
out = Gauss_mod.fit(y, pars, x=x)

print(out.fit_report())
plt.figure(figsize=(6,4))
plt.plot(x,y, 'rx')
plt.plot(x,out.best_fit, "b", label="FWHM = {:.3f}".format(out.values['fwhm']))
plt.title(f'Run {run_number}, full photon peak, sectors aligned')
plt.legend(loc="best")
plt.xlabel("dldTimeSteps [step]")
plt.show()

As we can see from the result of the last fit, after sector alignment, we have improved the photon peak width by 0.058 steps.

The same check can be done for every single sector to see/check that all sectors were properly corrected in time by their difference from the 0 sector.

In [ ]:
plt.figure(figsize=(6,4))
for i, item in enumerate(ph_peak_align):
    x=ph_peak_align['dldTimeSteps']
    y=item
    pars = Gauss_mod.make_params(amplitude=800.0, center=2390, sigma=1)
    out = Gauss_mod.fit(y, pars, x=x)
    Center = 2388.984276411258
    Diff = "{:.3f}".format(Center - out.values['center'])
    FWHM = "{:.3f}".format(out.values['fwhm'])
    item.plot(label=f'S{i}={Diff}, FWHM = {FWHM} step')
    plt.title(f'Run {run_number}, individual sectors, aligned')
    plt.legend()
plt.show()

Now we can make an ns conversion and do a fitting procedure again to see, e.g. DLD time resolution.

In [ ]:
sp_ph_peak.append_tof_ns_axis()

In [ ]:
sp_ph_peak.dataframe[["dldTimeSteps", "dldTime", "dldSectorID"]].head()

In [ ]:
axes = ['dldSectorID', 'dldTime']
ranges = [[0,8], [390,397]]
bins = [8,350]
res_ph_peak_ns = sp_ph_peak.compute(bins=bins, axes=axes, ranges=ranges)

plt.figure()
res_ph_peak_ns.sum(('dldSectorID')).plot()
plt.show()

In [ ]:
Gauss_mod = GaussianModel()

x=res_ph_peak_ns['dldTime']
y=res_ph_peak_ns.sum('dldSectorID')

pars = Gauss_mod.make_params(amplitude=340.0, center=393.2, sigma=0.19)
# pars = Gauss_mod.guess(y, x=x)
out = Gauss_mod.fit(y, pars, x=x)

print(out.fit_report())
plt.figure()
plt.plot(x,y, 'rx')
plt.plot(x,out.best_fit, "b", label="FWHM = {:.3f} ns".format(out.values['fwhm']))
#plt.title(f'Run {runs}, full photon peak')
plt.legend(loc="best")
plt.xlabel("dldTime [ns]")
plt.title(f'Run {run_number}, all sectors aligned')
plt.show()